In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
import keras

In [2]:
def model(input_shape):
    x = layers.Input(shape=(input_shape,))

    embedding = layers.Embedding(5, 2, mask_zero=True, input_length=1)
    flatten = layers.Flatten()
    dense = layers.Dense(units=1, activation='softmax')
    
    e = embedding(x)
    p = flatten(e)
    y = dense(p)
    model = keras.Model(x, y)
    return model

In [4]:
m = model(1)
m.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1, 2)              10        
_________________________________________________________________
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [8]:
#x = [[1, 4, 3, 1], [1, 2, 2, 1], [4, 1, 1, 0]]
x = [[1], [2], [4], [3], [5]]
y = [1, 1, 0, 0, 1]

In [9]:

optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['AUC']
epochs = 20

m.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [10]:
m.fit(x, y)

InvalidArgumentError:  indices[1,0] = 5 is not in [0, 5)
	 [[node model_1/embedding_1/embedding_lookup (defined at <ipython-input-10-46f86a1c0ae6>:1) ]] [Op:__inference_train_function_1044]

Errors may have originated from an input operation.
Input Source operations connected to node model_1/embedding_1/embedding_lookup:
 model_1/embedding_1/embedding_lookup/706 (defined at c:\Users\user\anaconda3\envs\SportScience\lib\contextlib.py:81)

Function call stack:
train_function


In [40]:
model = keras.models.Sequential()
model.add(layers.Embedding(5, 2))

input_array = [[1], [2], [3]]

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)

In [41]:
print(input_array)
print(output_array)

[[1], [2], [3]]
[[[0.03754575 0.04420992]]

 [[0.02595857 0.01621464]]

 [[0.03411372 0.00356693]]]


In [42]:
output_array.shape

(3, 1, 2)

In [45]:
output_array.resize([output_array.shape[0], output_array.shape[2]])

In [46]:
output_array

array([[0.03754575, 0.04420992],
       [0.02595857, 0.01621464],
       [0.03411372, 0.00356693]], dtype=float32)